time-dependent, one-dimensional, heat condunction equation: $\frac{\partial T}{\partial t}-\alpha \frac{\partial^2 T}{\partial x^2} = 0$, $x\in [0,L]$

BCs: $T(0,t)=90$, $T(L,t)=20$

Initial Condition: $T(x,0)=20$ for $x>0$, $T(0,0)=T_{\infty}=90$

$L=0.04, \rho=900, k=0.55, C_p=3800, \alpha=\frac{k}{\rho C_p}$

Analytical solution: $u(\xi, \tau) = \sum_{n=1}^{\infty}(-1)^n\frac{2cos(n\pi)}{n\pi} exp(-n^2 \pi^2 \tau) sin(n\pi\xi) + \xi$, where $u = \frac{T - T_{\infty}}{T_L - T_{\infty}}$, $\xi = \frac{x}{L}$, $\tau = t \alpha / L^2$

Dimensionless BCs & Init: $u(0, \tau) = 0$, $u(1, \tau) = 1$; $u(\xi, 0) = 1$ for $\xi > 0$ and $u(0, 0) = 0$

In [ ]:
import torch
import numpy as np
import torch.utils.data
import torch.nn.functional as F
from torch.autograd import Variable
import timeit
import torch
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.ndimage import convolve


In [ ]:

# setup for parameters
lag=8
n = 100
L = 0.04
rho = 900
k = 0.55
Cp = 3800
lr=0.0005
alpha = k/(rho*Cp)
alpha
T0             = np.zeros(100, dtype=float)
T0[1:]         = 20.
T0[0]          = 90
batch_size=10

In [ ]:
#fd integrator using ee method
def int_ee(T0, steps):

    T1             = T0.copy()  

    do_me          = np.ones_like(T0, dtype=bool)
    do_me[[0, -1]] = False    #  keep the boundaries of your bounding box fixed

    dt              = 0.08
    dx             = L/(n+1)
    for i in range(steps):
        coff       = dt*alpha/dx/dx

        Ttemp      = T1 + coff*(np.roll(T1, +1) + np.roll(T1, -1) - 2*T1)
        T1[do_me]  = Ttemp[do_me]
        
    return T1

In [ ]:
def fetch_batch():
    num=np.random.randint(10000, size=batch_size) 
    b=[]
    tar=[]
    print(num)
    for n in num:
        t=int_ee(T0, n)
        tar.append(t)
        X=np.zeros((lag,len(T0)))
        for i in range(len(X)):
            X[i]=t.copy()
        b.append(X)
    
    b=np.asarray(b)
    tar=np.asarray(tar)
    b = torch.from_numpy(b).type(torch.FloatTensor)
    
   
    return b,tar


b,t= fetch_batch()
k=int_ee(t, 3000)
print(k.shape)

In [ ]:

class NN(torch.nn.Module):
    def __init__(self,k):
        super(NN, self).__init__()
        self.k=k
    
        # encoding part
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv1d(k, 16, 5, stride=1),
            torch.nn.BatchNorm1d(16),
            torch.nn.Tanh()
        )
        
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv1d(16, 32, 5, stride=2),
            torch.nn.BatchNorm1d(32),
            torch.nn.Tanh()
        )
        
        # dense part
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv1d(32, 32, 1),
            torch.nn.BatchNorm1d(32),
            torch.nn.Tanh()
        )
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv1d(32, 32, 1),
            torch.nn.BatchNorm1d(32),
            torch.nn.Tanh()
        )
        # decoding part
        self.conv5 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(64, 16, 1, stride=2),
            torch.nn.BatchNorm1d(16),
            torch.nn.Tanh()
        )
        
        self.conv6 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(16, 1, 1, stride=2),
           
            torch.nn.Tanh()
        )
        
        self.conv7 = torch.nn.Sequential(
            torch.nn.ConvTranspose1d(1, 1, 6,stride=5),
            torch.nn.Tanh()
        )
    
        
       
    def forward(self, u):
        
        u = self.conv1(u)
        #print('1:',u.shape)
        u = self.conv2(u)
        uin=u
        
        u = self.conv3(u)
        
       
        u = self.conv4(u)
        u=torch.cat([uin,u],dim=1)
      
       
        u = self.conv5(u)
        #print('5:',u.shape)
        u = self.conv6(u)
        #print('6:',u.shape)
        u= self.full1(u)
        #print('f:',u.shape)
        u= torch.squeeze(u)
        #print('r:',u.shape)
        return u
    
    def mse(self,u,u_num):
        loss = torch.mean((u - u_num)**2)
        return loss
    
    def addele(self,X,u):
  
        new_X=X.clone()
        
        new_X = Variable(new_X, requires_grad=False)
        
  
        new_X[:,0,:]=u
        new_X[:,1:,:]=X[:,:-1,:]
        return new_X
    
    def train(self, N,p,Tmax,lr):
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        L=0
        tsteps = np.linspace (p, Tmax, N)
        L_trac=[]
        for epoch in range(N):
            
           
            decayRate = 0.9
            scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
            
            print("new epoch-----------------------------",epoch)
            
            X,u0=fetch_batch()
            X=Variable(X, requires_grad=False)
            
            T=tsteps[epoch]
        
            for i in range(int(T)):
                
                u=self.forward(X)
            
                u_num= int_ee(u0, i*10)
                u_num = torch.from_numpy(u_num).type(torch.FloatTensor)
                u_num = Variable(u_num, requires_grad=False)
                L=L+self.mse(u,u_num)
    
                if i%p==0:
                    print("loss ")
                    print(L)
                    if i+p>int(T)-2:
                        
                        L_trac.append(L.data.numpy())
                  
                    L.backward(retain_graph=True)
                    optimizer.step()
                    optimizer.zero_grad()
                    L=0
                X=self.addele(X,u)
            scheduler.step()
        return L_trac
    def addele_p(self,X,u):
  
        new_X=X.clone()

        new_X = Variable(new_X, requires_grad=False)


        new_X[:,0]=u
        new_X[:,1:]=X[:,:-1]
        return new_X
        
    def predict(self,u0,Tmax):
            X = np.zeros((self.k,len(u0)))
            for i in range(len(X)):
                X[i]=u0
            X = torch.from_numpy(X).type(torch.FloatTensor)
            X = Variable(X, requires_grad=False)
            shape=X.shape
            X = X.view(-1,shape[0],shape[1])
            out=[]
            out.append(np.asarray(u0))
            for n in range(Tmax):
                u=self.forward(X)
                u[0]=90
                u[-1]=20
                ele=torch.squeeze(u)
                
                ele=ele.data.numpy()
                if n% 1000==0:
                    print(ele)
                out.append(ele.copy())
                X=self.addele_p(X,u)
            return out


In [ ]:
network=NN(8)

In [ ]:
trac=network.train(50,2,200,lr)


In [ ]:
plt.plot(trac)
plt.xlabel('epoch')
plt.ylabel('mse loss')
plt.title('training loss vs epoch')
plt.savefig('train.png')

In [ ]:
def addele_o(X,u):
    print(X)
    new_X=X.clone()

    new_X = Variable(new_X, requires_grad=False)


    new_X[:,0]=u
    new_X[:,1:]=X[:,:-1]
    print(new_X)
    return new_X


def predict_o(model,u0,Tmax):
        X = np.zeros((model.k,len(u0)))
        for i in range(len(X)):
            X[i]=u0
        X = torch.from_numpy(X).type(torch.FloatTensor)
        X = Variable(X, requires_grad=False)
        shape=X.shape
        X = X.view(-1,shape[0],shape[1])
        out=[]
        out.append(np.asarray(u0))
        for n in range(Tmax):
            
            u=model.forward(X)
            u[0]=90
            u[-1]=20
           
            ele=torch.squeeze(u)

            ele=ele.data.numpy()
           
                
            out.append(ele.copy())
            X=addele_o(X,u)
            
        return out


In [ ]:
out=predict_o(network,T0,2000)


In [ ]:

plt.plot(range(100),out[500],label='prediction')
plt.plot(range(100),ref[5000],label='numerical')
plt.xlabel('points along the line')
plt.ylabel('T')
plt.title('prediction of 5000 time step')
plt.legend()
plt.savefig('p2.png')

plt.show()

In [ ]:
#ref
def int_ee_ref(T0, steps):

    T1             = T0.copy()  

    do_me          = np.ones_like(T0, dtype=bool)
    do_me[[0, -1]] = False    #  keep the boundaries of your bounding box fixed
    
    dt              = 0.08
    dx             = L/(n+1)
    out=[]
    out.append(T0)
    for i in range(steps):
        coff       = dt*alpha/dx/dx

        Ttemp      = T1 + coff*(np.roll(T1, +1) + np.roll(T1, -1) - 2*T1)
        T1[do_me]  = Ttemp[do_me]
        out.append(T1.copy())
        
    return out

In [ ]:
ref=int_ee_ref(T0, 20000)
ref=np.asarray(ref)

In [ ]:
for i in range(20000):
    if i % 1000==0:
        plt.plot(range(100),ref[i])
plt.xlabel('points along the line')
plt.ylabel('T')
plt.title('numerical solution')
plt.savefig('p1.png')

plt.show()

In [ ]:
k=T0.copy()
for i in range(2000):
    if i % 100==0:
        out=predict_o(network,T0,i)
        k=out[-1]
        plt.plot(range(100),out[-1],label='predict') 
        
#plt.plot(range(100),ref[20000],label='numerical')        
plt.xlabel('points along the line')
plt.ylabel('T')
plt.title('prediction')
plt.savefig('p2.png')


plt.show()

In [ ]:
torch.save(network.state_dict(),"C:/Users/manqi/mymodel")

In [ ]:
from IPython.display import clear_output

for i in range(10):
    clear_output(wait=True)
    print(i, flush=True)